# Data Mining for Business Analytics

## Similarity, Neighbors

Spring 2019 - Prof. George Valkanas

Material based on content courtesy of Prof. Foster Provost

***


## Clustering

We already saw how we can use _similarity_ between two instances to produce recommendations, even in the absence of a target variable.

_Similarity_ is also a key element in **clustering**, i.e., the generation of _natural groups_ of our instances. We have already seen that different similarity measures result in different _rankings_ of the same set of instances. With that in mind, the _similarity_ that we use greatly affects the result of our **clustering** algorithms.


Below, we discuss 2 different clustering techniques:
* $k$-Means
* Hierarchical Clustering

Through our discussion, you need to be able to understand:
1. How **clustering** differs from **classification**
1. What is _the result_ of a clustering algorithm
1. How $k$-Means works
1. How hierarchical clustering works

***

In [ ]:
# Import the libraries we will be using
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import train_test_split

from scipy.spatial.distance import pdist, squareform
from scipy.cluster.hierarchy import linkage, dendrogram

from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from scipy.spatial import distance

import math
import pyproj as proj

%matplotlib inline
import matplotlib.pylab as plt
plt.rcParams['figure.figsize'] = 10, 8



## KMeans

The "classic" approach for findings clusters is via the **$k$-Means algorithm**, which will find a set of $k$ clusters.

The value $k$ is a **parameter** to the model, i.e. **we** must provide the number of clusters that we expect the algorithm to find.

**Question:** _What_ is a good $k$ value for the algorithm?


Here's a video of how the algorithm works: https://www.youtube.com/watch?v=BVFG7fd1H30


The $k$-Means algorithm is implemented in all major (self-respecting) data mining libraries. It's also available under **sklearn.cluster**.



In [ ]:
# Let's read our Whiskey data again
data = pd.read_csv("data/scotch.csv")
data = data.drop([u'age', u'dist', u'score', u'percent', u'region', u'district', u'islay', u'midland', u'spey', u'east', u'west', u'north ', u'lowland', u'campbell', u'islands'], axis=1)

In [ ]:

k_clusters = 6

## Fit clusters like in our previous models/transformations/standarization 
## (e.g. Logistic, Vectorization,...)

model = KMeans(k_clusters)
model.fit(data)


What clusters do we get? Let's get "predictions" of the model:


In [ ]:
print ("Records in our dataset (rows): ", len(data.index))
print ("Then we predict one cluster per record, which means length of: ", len(model.predict(data)) )


In [ ]:
data.index

In [ ]:
clusters = model.predict(data)

clusters

In [ ]:
pd.DataFrame(list(zip(data.index,model.predict(data))), columns=['Whiskey','Cluster_predicted']) [0:10]

Let's put each cluster into its own column!


In [ ]:

cluster_listing = {}
for cluster in range(k_clusters):
    cluster_listing['Cluster ' + str(cluster)] = [''] * 109
    where_in_cluster = np.where(clusters == cluster)[0]
    cluster_listing['Cluster ' + str(cluster)][0:len(where_in_cluster)] = data.index[where_in_cluster]

# Print clusters
pd.DataFrame(cluster_listing).loc[0:np.max(np.bincount(clusters)) - 1,:]


## Hierarchical Clustering

There are different ways to find similar groups.  One very common method is Hierarchical Clustering.

First let's look at a simple example to illustrate.  Given a set of records (A-F) with two features, we can visualize them on a 2 dimensional surface.  Clustering proceeds as follows.  First consider each point to be its own cluster.  Then, iteratively, group together the closest two clusters.  In the figure, circles were drawn in order of grouping.  The second diagram is a visualization of the hierarchy of groupings, called a "dendrogram."  You can clip it at any point, vertically, and get "the best" clustering for a certain number of groups.


<img src="images/cutting.png" height=40% width=40%>

Here is a visualization of a part of the dendrogram for the whiskey clustering in the book:

<img src="images/cross_section.png" height=70% width=70%>

***



### Let's examine the dendrogram(s) for our data, we'll be using the library: **scipy.cluster.hierarchy**

In [ ]:
# This function gets pairwise distances between observations in n-dimensional space.
dists = pdist(data, metric="cosine")

# This scipy's function performs hierarchical/agglomerative clustering on the condensed distance matrix y.
links = linkage(dists, method='average')

# Now we want to plot those 'links' using "dendrogram" function
plt.rcParams['figure.figsize'] = 32, 16

den = dendrogram(links)

plt.xlabel('Samples',fontsize=18)
plt.ylabel('Distance',fontsize=18)
plt.xticks(rotation=90,fontsize=16)
plt.show()


We can use other measures:

In [ ]:
# This function gets pairwise distances between observations in n-dimensional space.
dists = pdist(data, metric="euclidean")

# This scipy's function performs hierarchical/agglomerative clustering on the condensed distance matrix y.
links = linkage(dists, method='average')

# Now we want to plot those 'links' using "dendrogram" function
plt.rcParams['figure.figsize'] = 32, 16

den = dendrogram(links)

plt.xlabel('Samples',fontsize=18)
plt.ylabel('Distance',fontsize=18)
plt.xticks(rotation=90,fontsize=16)
plt.show()


#### Question

Clusters **do not** come with predetermined labels. Can you think of an approach to "characterize them" ? Can you think of an _automated approach_ to characterize them?
***